Reference: [Build a Retrieval Augmented Generation (RAG) App](https://python.langchain.com/v0.2/docs/tutorials/rag/)

In [1]:
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv() # invoke langsmith

llm = ChatOpenAI(model="gpt-3.5-turbo")

## Build vector database

In [ ]:
# import os

# from langchain import hub
# from langchain_community.document_loaders import UnstructuredMarkdownLoader, PyPDFLoader
# from langchain_community.vectorstores import Qdrant
# from langchain_openai import OpenAIEmbeddings
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# embedding_cache_path = "embedding_cache"
# db_collection_names = ["CNS16190_md", "CNS16190_md_only_provision"]
# document_root_path = "docs"
# documents = ["CNS16190-zh_TW.md", "CNS16190-zh_TW_only_provision.md"]

# collection = db_collection_names[0]
# if os.path.isdir(os.path.join(embedding_cache_path, "collection", collection)):
#     # database already exists, load it
#     print("Loading existing database")
#     vectorstore = Qdrant.from_existing_collection(
#         embedding=OpenAIEmbeddings(),
#         path=embedding_cache_path,
#         collection_name=collection,
#     )
# else:
#     # database does not exist, create it
#     print("Creating new database")
#     loader = UnstructuredMarkdownLoader(
#         os.path.join(document_root_path, documents[0]), mode="elements"
#     )
#     doc = loader.load()

#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#     splits = text_splitter.split_documents(doc)

#     vectorstore = Qdrant.from_documents(
#         splits,
#         embedding=OpenAIEmbeddings(),
#         path=embedding_cache_path,
#         collection_name=collection,
#     )

# retriever = vectorstore.as_retriever()

In [2]:
import os

from langchain import hub
from langchain_community.document_loaders import UnstructuredMarkdownLoader, PyPDFLoader
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embedding_cache_path = "embedding_cache"
db_collection_names = ["CNS16190-zh_TW_only_provision", "ts_103701v010101p_without_content"]
document_root_path = "docs"
documents = ["CNS16190-zh_TW_only_provision.pdf", "ts_103701v010101p_without_content.pdf"]

# 創建一個空列表來保存所有分割後的文檔
all_splits = []

# 加載和分割每個文檔
for idx, document in enumerate(documents):
    print(f"Processing document: {document}")
    if document.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(document_root_path, document))
    elif document.endswith(".md"):
        loader = UnstructuredMarkdownLoader(os.path.join(document_root_path, document), mode="elements")
    else:
        continue

    doc = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(doc)
    all_splits.extend(splits)

# 將所有分割後的文檔添加到同一個 Qdrant 集合中
vectorstore = Qdrant.from_documents(
    all_splits,
    embedding=OpenAIEmbeddings(),
    path=embedding_cache_path,
    collection_name="combined_collection",
)

# 設置檢索器
retriever = vectorstore.as_retriever()

print("All documents have been processed and added to the Qdrant collection.")


Processing document: CNS16190-zh_TW_only_provision.pdf
Processing document: ts_103701v010101p_without_content.pdf
All documents have been processed and added to the Qdrant collection.


In [3]:

prompt = hub.pull(
    "rlm/rag-prompt"
)  # https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=07050030-99d2-5925-9ec6-8a8fb67164d3

prompt.messages[0].prompt.template="你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：{context} \n問題：{question} \n答案："
prompt


ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：{context} \n問題：{question} \n答案：'))])

## Build rag chain

In [4]:
import pandas as pd


# concat all docs as a string
def format_docs(docs):
    retrieval_docs = "\n\n".join(doc.page_content for doc in docs)
    df = pd.read_csv("docs/test_data_from_GoogleSheet.csv")
    test_data = "\n\n測試資料範例：\n\n"
    for idx in range(3):
        row = df.iloc[idx]
        test_data += f"控制措施{row['provision']}，detail:{row['detail']}，question:{row['Test Scenario']}，answer:{row['answer']}\n\n"

    result = retrieval_docs + test_data
    print(result)
    return result



In [5]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)
# This is equivalent to following without using format_docs():
# question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# the path of the rag_chain:
# 1. get "question" from rag_chain.invoke()
# 2. search the most relevant context from the retriever
# 3. concat the docs from retriever in format_docs() as a string
# 4. format the string as a prompt
# 5. query the prompt to the llm

## Ask llm

In [6]:
df = pd.read_csv("docs/test_data_from_GoogleSheet.csv")

df

,provision,status,support,detail,Test Scenario,answer,reason,result
0,5.1-1,MC,NaN,如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的...,問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置...,使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或...,NaN,NaN
1,5.1-3,M,NaN,用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。,與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通...,本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TL...,NaN,NaN
2,5.2-1,M,NaN,製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資...,漏洞揭露政策明確規定了安全研究人員和其他人報告問題的流程。 此類政策可以根據需要進行更新，以...,通過D-Link 網站（https://support.dlink.com/ReportVu...,NaN,NaN
3,5.3-13,M,NaN,製造商應以對使用者易於理解、透明的方式公開定義的支援期限。,消費者在購買設備時，希望能明確說明該期限的軟體更新支援。問題：1. 客戶如何獲知定義的支援期...,產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。,NaN,NaN
4,5.3-16,M,NaN,消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。,問題：1. 使用者如何驗證設備型號名稱（透過設備上的標籤或透過實體介面）？,此產品的型號標籤上有相關的生產編號資訊。,NaN,NaN
5,5.4-1,M,NaN,持久存儲中的敏感安全參數應由設備安全的存儲。,安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉...,所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。,NaN,NaN
6,5.4-3,M,NaN,關鍵安全參數不得硬編碼於使用設備軟體原始碼中。,設備和應用程式的逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼。注意：此類憑證...,D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded c...,NaN,NaN
7,5.4-4,M,NaN,用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個...,為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果...,D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\n管理...,NaN,NaN
8,5.5-1,M,NaN,消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。,安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,NaN,NaN
9,5.5-5,M,NaN,設備功能只有在認證存取之後允許透過網路介面進行配置安全相關修改。例外情況是設備所依賴的網路服...,注意：例外的協定包括 ARP、DHCP、DNS、ICMP 和 NTP。範例：與安全性相關的變...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,NaN,NaN


### One row testing

In [7]:
row = df.iloc[6]
query = f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
print(
    f"控制措施 {row['provision']}\n\n【detail】\n{row['detail']}\n\n【question】\n{row['Test Scenario']}\n\n【answer】\n{row['answer']}"
)

print(f"\n【Test scenario】\n{row['Test Scenario']}")


控制措施 5.4-3

【detail】
關鍵安全參數不得硬編碼於使用設備軟體原始碼中。

【question】
設備和應用程式的逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼。注意：此類憑證通常可以在原始程式碼中找到，這是眾所周知的不良做法。注意：也用於模糊或加密此硬編碼訊息的簡單混淆方法可能會被輕易破壞。這些憑證還可以是：- 允許在遠端服務中使用安全敏感功能的 API 金鑰，或者- 用於裝置通訊協定安全的私鑰。問題：1. 是否使用了軟體原始碼中硬編碼的關鍵安全參數？

【answer】
D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded critical security parameters 在程式中

【Test scenario】
設備和應用程式的逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼。注意：此類憑證通常可以在原始程式碼中找到，這是眾所周知的不良做法。注意：也用於模糊或加密此硬編碼訊息的簡單混淆方法可能會被輕易破壞。這些憑證還可以是：- 允許在遠端服務中使用安全敏感功能的 API 金鑰，或者- 用於裝置通訊協定安全的私鑰。問題：1. 是否使用了軟體原始碼中硬編碼的關鍵安全參數？


In [10]:
rag_chain.invoke(f"{query}\n\n根據控制措施和detail在ETSI TS 103 701中找出與控制措施最相關Test group，\
                 並生成測試流程來供測試者驗證answer 是否真的符合控制措施detail要求。")

控制措施5.4-
2：對安全目的，於裝置中使用硬編碼唯一之每裝置識別資訊時，應以防止藉由實
體、電氣或軟體等方式竄改的方式實作。
例 
4.用於網路接取對裝置唯一之主金鑰儲存於遵循相關ETSI標準的UICC中(例：參照ETSITS102221[25])
。
控制措施5.4-3：不應使用裝置軟體原始碼中硬編碼之關鍵安全參數。
裝置及應用程式之逆向工程，能輕易發現軟體中硬編碼的使用者名稱及通行碼等信符。此等信符
亦可為容許於遠端服務中，使用安全敏感功能性之API金鑰，或為裝置用以通訊的協定安全中所使
用之私鑰。此種信符通常可於原始碼內發現，此係公認之不良實務作法。用以隱藏或加密此硬編
碼資訊之簡單混淆方法亦能輕易破解。
控制措施5.4-
4：用於軟體更新之完整性及真確性的核對，以及用於保護與裝置軟體中相關聯服
務之通訊的任何關鍵安全參數，應為每裝置唯一，並應以降低對各類別裝置自動化
攻擊的機制產生。
例 5.於同一產品類別之各裝置上部署不同的對稱金鑰，用以產生並查證軟體更新之訊息鑑別碼。
例 6.裝置使用製造者之公鑰查證軟體更新。此並非關鍵安全參數，且無需為每裝置唯一者。
為裝置提供唯一之關鍵安全參數，有助於保護軟體更新的完整性及真確性，以及裝置與相關聯服
務之通訊。若使用全域關鍵安全參數，則其揭露可能導致對其他IoT裝置之大規模攻擊，諸如建立
殭屍網路。

5.5 安全通訊
控制措施5.5-
1：消費者IoT裝置應使用最佳實務密碼學安全通訊。安全控制措施之適切性及最
佳實務密碼學的使用，取決於許多因素，包括使用全景。隨著安全不斷演進，很難
提供關於密碼學或其他安全措施之規定性建議，而無此種建議很快過時的風險。
控制措施5.5-
2：消費者IoT裝置宜使用經審查或經評估之實作，交付網路功能性及安全功能性
，特別是密碼學領域。
審查及評估可能涉及獨立之內部或外部個體。
例 
1.開發及測試社群中散佈之軟體程式館、經驗證軟體模組，以及硬體設備加密服務提供者(諸如安
全元件及受信任執行環境)係全部經審查或評估。
控制措施5.5-3：密碼演算法及基元宜為可更新。
備考1. 此亦稱為〝密碼敏捷性〞。
對無法更新之裝置，重要的是裝置之預期壽命不超過裝置所使用密碼演算法的建議使用壽命(包括
金鑰長度)。
控制措施5.5-
4：宜僅於鑑別介面後，方可經由網路介面於初始化狀態存取裝置功能

AIMessage(content='根據控制措施5.4-3的要求，最相關的測試群組可能是針對軟體原始碼中是否硬編碼關鍵安全參數的測試群組。測試流程可能包括以下步驟：\n1. 檢查裝置的軟體原始碼，確保沒有硬編碼的關鍵安全參數。\n2. 進行靜態代碼分析，以確認是否有任何關鍵安全參數是硬編碼在程式碼中。\n3. 進行動態測試，模擬攻擊情境並檢查是否可以輕易取得硬編碼的安全參數。\n4. 確保裝置使用的是安全的方式來存儲和處理關鍵安全參數。\n5. 最終確認製造商的安全政策是否包括禁止在軟體原始碼中硬編碼關鍵安全參數的要求。\n\n這些步驟可以確保製造商是否遵循了控制措施5.4-3的要求，並確保裝置的安全性符合相關標準。', response_metadata={'token_usage': {'completion_tokens': 402, 'prompt_tokens': 5060, 'total_tokens': 5462}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-befeeb7f-084d-430c-82e9-267752a46ae2-0', usage_metadata={'input_tokens': 5060, 'output_tokens': 402, 'total_tokens': 5462})

### Batch test

In [11]:
queries = []
for row in range(len(df) - 3):
    row = df.iloc[row + 3]
    condition = (
        f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
    )
    query = f"{condition}\n\n根據控制措施和detail在ETSI TS 103 701中找出與控制措施最相關Test group，並生成測試流程來供測試者驗證answer 是否真的符合控制措施detail要求。"
    queries.append(query)
queries


['控制措施5.3-13，detail:製造商應以對使用者易於理解、透明的方式公開定義的支援期限。，answer:產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。\n\n根據控制措施和detail在ETSI TS 103 701中找出與控制措施最相關Test group，並生成測試流程來供測試者驗證answer 是否真的符合控制措施detail要求。',
 '控制措施5.3-16，detail:消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。，answer:此產品的型號標籤上有相關的生產編號資訊。\n\n根據控制措施和detail在ETSI TS 103 701中找出與控制措施最相關Test group，並生成測試流程來供測試者驗證answer 是否真的符合控制措施detail要求。',
 '控制措施5.4-1，detail:持久存儲中的敏感安全參數應由設備安全的存儲。，answer:所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。\n\n根據控制措施和detail在ETSI TS 103 701中找出與控制措施最相關Test group，並生成測試流程來供測試者驗證answer 是否真的符合控制措施detail要求。',
 '控制措施5.4-3，detail:關鍵安全參數不得硬編碼於使用設備軟體原始碼中。，answer:D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded critical security parameters 在程式中\n\n根據控制措施和detail在ETSI TS 103 701中找出與控制措施最相關Test group，並生成測試流程來供測試者驗證answer 是否真的符合控制措施detail要求。',
 '控制措施5.4-4，detail:用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個設備獨有，並且應具有降低對設備類別進行自動攻擊風險的機制。，answer:D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\n管理和設定通道始終透過 TLS v1.2 或更新的連線進行保護，chiper 套件符合 FIPS 14--3。\n伺服

In [12]:
results = rag_chain.batch(queries)

控制措施5.3-
14：對無法更新其軟體之受限制裝置，製造者宜以對使用者清晰透通的可存取方
式，公布未更新軟體之理由闡述、硬體更換支援的期間及方法，以及定義之支援期
間。
控制措施5.3-15：對無法更新軟體之受限制裝置，產品宜為可隔離且硬體可更換。
於某些情況下，裝置無法修補程式。對受限制裝置，需備妥更換計畫並清楚傳達予消費者。此計
畫通常詳細說明何時需更換技術，以及適用時，何時結束對硬體及軟體之支援等的排程。
控制措施5.3-
16：消費者IoT裝置之型號名稱應藉由於裝置上加標籤，或經由實體介面可清晰辨
識。
此通常經由邏輯介面與裝置通訊所執行，但其亦可為UI之一部分。
例 
13.裝置具報告型號名稱之HTTP(或適切時為HTTPS)API(於使用者鑑別後)。通常要求瞭解裝置之特定
名稱，以核對定義的軟體更新支援期間或軟體更新的可用性。
5.4 安全儲存敏感性安全參數
控制措施5.4-1：持久性儲存體中之敏感性安全參數應由裝置安全儲存。
安全儲存機制可用以保護敏感性安全參數。適切之機制包括由受信任執行環境(TEE)所提供的機制
，以及與硬體、安全元件(SE)或專用安全組件(DSC)相關聯之加密儲存，並於UICC(依ETSITR121905[
29]、ETSITS102221[25])上/於嵌入式UICC(依GSMASGP.22TechnicalSpecificationv2.2.1[26])上運行的軟
體之處理能力。
備考：此控制措施適用於持久性儲存體，但製造者亦可對記憶體中之敏感性安全參數實作類似作
法。
例 1.授權及接取許可之無線電頻率(例：LTE-m細胞式接取)所涉及的根金鑰儲存於UICC中。
例 2.使用受信任執行環境(TEE)儲存並存取敏感性安全參數之遠端控制門鎖。
例 3.無線恆溫器將無線網路之信符儲存於防竄改微控制器中，而非於外部快閃儲存體中。

控制措施5.4-
2：對安全目的，於裝置中使用硬編碼唯一之每裝置識別資訊時，應以防止藉由實
體、電氣或軟體等方式竄改的方式實作。
例 
4.用於網路接取對裝置唯一之主金鑰儲存於遵循相關ETSI標準的UICC中(例：參照ETSITS102221[25])
。
控制措施5.4-3：不應使用裝置軟體原始碼中硬編碼之關鍵安全參數。
裝置及應用程式之逆向工程，能輕易發現軟體中硬編碼的使用者名稱及通行碼等信符。此等

In [13]:
results

[AIMessage(content='根據控制措施5.3-13的要求，最相關的測試組織應該是與支援期限和公告相關的測試組織。測試流程可能包括以下步驟：\n1. 確認製造商是否在產品文檔或網站上清楚公開了支援期限的定義。\n2. 檢查製造商是否提前60天在產品網頁上公告產品生命維護週期結束的消息。\n3. 確認製造商是否提供易於理解的方式供使用者查詢支援期限。\n4. 測試者可以模擬產品生命維護週期結束前60天的情況，觀察製造商是否確實公告了相關信息。\n\n透過以上測試流程，測試者可以驗證製造商是否符合控制措施5.3-13中對支援期限公開透明的要求。', response_metadata={'token_usage': {'completion_tokens': 316, 'prompt_tokens': 5773, 'total_tokens': 6089}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b9f26291-91f8-4b72-bc7e-004b0aa102d1-0', usage_metadata={'input_tokens': 5773, 'output_tokens': 316, 'total_tokens': 6089}),
 AIMessage(content='根據控制措施5.3-16的要求，可以在ETSI TS 103 701中找到相關的Test group，並生成測試流程來驗證製造商提供的答案是否符合要求。這樣可以確保消費者物聯網設備的型號名稱能夠清晰可識別，並且符合相關標準和規範。', response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 5243, 'total_tokens': 5377}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-090b03f6-

In [14]:
question_list = []
for result in results:
    question_list.append(result.content)


data = {
    "provision": df["provision"].iloc[3:].tolist(),
    "detail": df["detail"].iloc[3:].tolist(),
    "qustion": question_list,
    "answer": df["answer"].iloc[3:].tolist(),
}

result_df = pd.DataFrame(data)
result_df

,provision,detail,qustion,answer
0,5.3-13,製造商應以對使用者易於理解、透明的方式公開定義的支援期限。,根據控制措施5.3-13的要求，最相關的測試組織應該是與支援期限和公告相關的測試組織。測試流...,產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。
1,5.3-16,消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。,根據控制措施5.3-16的要求，可以在ETSI TS 103 701中找到相關的Test g...,此產品的型號標籤上有相關的生產編號資訊。
2,5.4-1,持久存儲中的敏感安全參數應由設備安全的存儲。,根據提供的背景資料，控制措施5.4-1要求持久性儲存中的敏感性安全參數應由裝置安全儲存，並提...,所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。
3,5.4-3,關鍵安全參數不得硬編碼於使用設備軟體原始碼中。,根據控制措施5.4-3的要求，最相關的Test group可能是與軟體安全相關的Test g...,D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded c...
4,5.4-4,用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個...,根據提供的背景資料，控制措施5.4-4要求用於軟體更新的完整性和真實性檢查的關鍵安全參數應該...,D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\n管理...
5,5.5-1,消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。,根據提供的背景資料，我無法確定控制措施5.5-1最相關的Test group是哪一個，也無法...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...
6,5.5-5,設備功能只有在認證存取之後允許透過網路介面進行配置安全相關修改。例外情況是設備所依賴的網路服...,對不起，我無法回答這個問題，因為我不是資安專家。我建議諮詢專業人士以獲得準確的答案。感謝理解。,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...
7,5.5-7,消費者物聯網設備應保護透過遠端存取網路介面的關鍵安全參數的機密性。,根據背景資料中提到的控制措施5.5-7，消費者物聯網設備應保護透過遠端存取網路介面的關鍵安全...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...
8,5.5-8,製造商應遵循與設備相關的關鍵安全參數的安全管理流程。,根據控制措施5.5-8，detail中提到製造商應遵循與設備相關的關鍵安全參數的安全管理流程...,按照D-Link的秘鑰管理流程生成、存儲和傳輸安全參數。 安全參數隨機生成后，寫入到設備中，...
9,5.6-1,應禁用所有未使用的網路和邏輯介面。,根據控制措施5.6-1的要求，最相關的Test group可能是檢查未使用的網路和邏輯介面是...,有一個 LAN 的乙太網路、2.4G 和 5G Wi-Fi 接口，需要在設備使用過程中運行，...


In [15]:
question_data = {
    "provision": df["provision"].iloc[3:].tolist(),
    "created question": question_list,
    "provided question": df["Test Scenario"].iloc[3:].tolist(),
    "answer": df["answer"].iloc[3:].tolist(),
}

question_df = pd.DataFrame(question_data)
question_df

,provision,created question,provided question,answer
0,5.3-13,根據控制措施5.3-13的要求，最相關的測試組織應該是與支援期限和公告相關的測試組織。測試流...,消費者在購買設備時，希望能明確說明該期限的軟體更新支援。問題：1. 客戶如何獲知定義的支援期...,產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。
1,5.3-16,根據控制措施5.3-16的要求，可以在ETSI TS 103 701中找到相關的Test g...,問題：1. 使用者如何驗證設備型號名稱（透過設備上的標籤或透過實體介面）？,此產品的型號標籤上有相關的生產編號資訊。
2,5.4-1,根據提供的背景資料，控制措施5.4-1要求持久性儲存中的敏感性安全參數應由裝置安全儲存，並提...,安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉...,所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。
3,5.4-3,根據控制措施5.4-3的要求，最相關的Test group可能是與軟體安全相關的Test g...,設備和應用程式的逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼。注意：此類憑證...,D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded c...
4,5.4-4,根據提供的背景資料，控制措施5.4-4要求用於軟體更新的完整性和真實性檢查的關鍵安全參數應該...,為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果...,D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\n管理...
5,5.5-1,根據提供的背景資料，我無法確定控制措施5.5-1最相關的Test group是哪一個，也無法...,安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...
6,5.5-5,對不起，我無法回答這個問題，因為我不是資安專家。我建議諮詢專業人士以獲得準確的答案。感謝理解。,注意：例外的協定包括 ARP、DHCP、DNS、ICMP 和 NTP。範例：與安全性相關的變...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...
7,5.5-7,根據背景資料中提到的控制措施5.5-7，消費者物聯網設備應保護透過遠端存取網路介面的關鍵安全...,可以使用加密的通訊通道或有效負載加密來實現機密性保護。注意：加密保護通常是使用至少與傳輸的密...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...
8,5.5-8,根據控制措施5.5-8，detail中提到製造商應遵循與設備相關的關鍵安全參數的安全管理流程...,注意：對關鍵安全參數使用開放的、同儕審查的標準（通常稱為「金鑰管理」）受到強烈鼓勵。問題：當...,按照D-Link的秘鑰管理流程生成、存儲和傳輸安全參數。 安全參數隨機生成后，寫入到設備中，...
9,5.6-1,根據控制措施5.6-1的要求，最相關的Test group可能是檢查未使用的網路和邏輯介面是...,範例：預設情況下，無法從 WAN 存取應從 LAN 存取的管理 UI。注意：邏輯介面 (LI...,有一個 LAN 的乙太網路、2.4G 和 5G Wi-Fi 接口，需要在設備使用過程中運行，...


In [16]:
question_df.to_csv("created_questions.csv", index=False, encoding="utf-8-sig")